In [85]:
"""

Baseline submission for Random Acts of Pizza
Michael Alexander

"""

%matplotlib inline

import json
import pandas
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

In [74]:
#Loading in initial json data from kaggle

path = 'C:/Users/malexander/Dropbox/MIDS/W207/Final_Project/Data/RAOP/'
train_file = 'train.json'
test_file = 'test.json'

#loading json files to dataframes
train_json = json.load(open(path+train_file))
test_json = json.load(open(path+test_file))

train_df = pandas.io.json.json_normalize(train_json)
test_df = pandas.io.json.json_normalize(test_json)

#Creating label, true/false
train_label_df = train_df.requester_received_pizza

#Subset of columns in train that are available in test
train_df = train_df[list(test_df)]

#Converting to np arrays
train_base = np.array(train_df.values)
test_base = np.array(test_df.values)
train_labels_base = np.array(train_label_df.values)

#Shuffling data and splitting into test and dev for training
np.random.seed(0)
shuffle = np.random.permutation(np.arange(train_base.shape[0]))
train_base, train_labels_base = train_base[shuffle], train_labels_base[shuffle]

#Only using numeric variables for now
numeric_variables = [4,5,6,7,8,9,10,12,13]

train_data, train_labels = train_base[:3000,numeric_variables], train_labels_base[:3000]
dev_data, dev_labels = train_base[3000:,numeric_variables], train_labels_base[3000:]
test_data = test_base[:,numeric_variables]

In [82]:
# Training decision tree, random forest, and ada boost models

dt = DecisionTreeClassifier(criterion="entropy", splitter="best", random_state=0)
dt.fit(train_data, train_labels)

print 'Accuracy (a decision tree):', dt.score(dev_data, dev_labels)

rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_data, train_labels)

print 'Accuracy (a random forest):', rfc.score(dev_data, dev_labels)

abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=100, learning_rate=0.1)

abc.fit(train_data, train_labels)
print 'Accuracy (adaboost with decision trees):', abc.score(dev_data, dev_labels)

Accuracy (a decision tree): 0.643269230769
Accuracy (a random forest): 0.728846153846
Accuracy (adaboost with decision trees): 0.75


In [88]:
# KNN test

for i in [10,20,30,40,50,100]:    
    
    knn_test = KNeighborsClassifier(n_neighbors = i)

    knn_test.fit(train_data, train_labels)
    
    print 'k = ' + str(i) + ' accuracy: ' + str(knn_test.score(dev_data,dev_labels))  

knn_predictions = knn_test.predict(dev_data)

print 'Classification Report:'
print(classification_report(dev_labels, knn_predictions))

k = 10 accuracy: 0.738461538462
k = 20 accuracy: 0.746153846154
k = 30 accuracy: 0.746153846154
k = 40 accuracy: 0.746153846154
k = 50 accuracy: 0.746153846154
k = 100 accuracy: 0.746153846154
Classification Report:
             precision    recall  f1-score   support

      False       0.75      1.00      0.85       776
       True       0.00      0.00      0.00       264

avg / total       0.56      0.75      0.64      1040



In [90]:
# Bernoulli Naive Bayes test

B_NB_model = BernoulliNB()

B_NB_model.fit(train_data, train_labels)

print 'Accuracy of the Bernoulli Naive Bayes model: ' + str(B_NB_model.score(dev_data, dev_labels))

B_NB_predictions = B_NB_model.predict(dev_data)

print 'Classification Report:'
print(classification_report(dev_labels, B_NB_predictions))

Accuracy of the Bernoulli Naive Bayes model: 0.698076923077
Classification Report:
             precision    recall  f1-score   support

      False       0.76      0.86      0.81       776
       True       0.35      0.22      0.27       264

avg / total       0.66      0.70      0.67      1040



In [73]:
# Creating test predictions

test_predictions = abc.predict(test_data)

test_request_ids = test_base[:,1]

# Converting predictions to binary
test_predictions_int = []

for i in range(len(test_predictions)):
    test_predictions_int.append(int(test_predictions[i]))

# Exporting submission csv
submission_df = pandas.DataFrame({"request_id" : test_request_ids, "requester_received_pizza" : test_predictions_int})
submission_df.to_csv(path+"Submission.csv", index=False)
